<a href="https://colab.research.google.com/github/sinajahangir/Foundation-Models/blob/main/TimesFM_FloodForecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is developed to showcase the benefits of TimesFM (https://github.com/google-research/timesfm/tree/master), a timeseries foundation model for zero-shot flood (water level and discharge) forecasting.

The code/notebook can be used for both streamflow and water level forecasting. This setting
can be set using the var_int='q' or 'wl'.

The processed data used in this notebook can be found at:
https://github.com/sinajahangir/HYSETS-Data-Process

The example provided here is associated with gauge 02GA014 in Ontario, CA.

#**Libraries**

##Instalation

In [ ]:
!pip install timesfm


##Import libraries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import timesfm
import numpy as np
import os
from google.colab import drive

 See https://github.com/google-research/timesfm/blob/master/README.md for updated APIs.
Loaded PyTorch TimesFM, likely because python version is 3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0].


#**Initialisation**


##Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

'''
This can take some time if drive is full
'''

Mounted at /content/drive


'\nThis can take time if drive is full\n'

#**Data**

In [ ]:
# set variable of interest. Either water level 'wl' or discharge 'q' for this problem
var_int='q'

In [ ]:
# Load the data
portion_='Train' #'Train' or 'Test'
data_file = f'/content/drive/MyDrive/TimesFM/Alldata_{portion_}_st_02GA014.csv' #change this
df = pd.read_csv(data_file)
file_name = os.path.basename(data_file)

In [ ]:
df['date'] = pd.to_datetime(df['date'])
df['unique_id'] = 1
df['ds'] = df['date']
time_series_df = df[['unique_id', 'ds', var_int]]

#**Zero Shot TimesFM**

##Initialisation

In [ ]:
# Initialize TimesFM model with pretrained weights
context_len=352 #lookback period. Should be a multiply of 32
tfm = timesfm.TimesFm(
      hparams=timesfm.TimesFmHparams(
          backend="gpu",
          per_core_batch_size=32,
          horizon_len=1,
          num_layers=50, #works very well
          use_positional_embedding=False, #works very well
          context_len=context_len,
      ),
      checkpoint=timesfm.TimesFmCheckpoint(
          huggingface_repo_id="google/timesfm-2.0-500m-pytorch"),
  )

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

##TimesFM

In [ ]:
# Prepare results list to store forecasted values
results = []

# Rolling forecast: predict 1 day at a time, advancing by 1 day in each loop
# last i: len(time_series_df)-context_len+context_len-1
for i in range(len(time_series_df)-context_len):  # Ensure there's enough data for 3-day forecasts
    # Forecast the next day
    forecast_df = tfm.forecast_on_df(
        inputs= time_series_df.iloc[:context_len+i,:],
        freq="d",  # daily input data
        value_name=var_int,
        num_jobs=-1,
    )


    # Append to results with corresponding observation for the first day of each period
    results.append({
        'date': time_series_df.iloc[context_len+i]['ds'], #it is import no note timesfm produces forecast(s) for date+forecast horizon
        'median':forecast_df['timesfm'].iloc[0],
        '10th':forecast_df['timesfm-q-0.1'].iloc[0], #10th percentile
        '90th':forecast_df['timesfm-q-0.9'].iloc[0], #90th percentile
        'obs': time_series_df.iloc[context_len+i][var_int]  # Observed value on the first day of the forecast period
    })

# Convert results list to DataFrame
results_df = pd.DataFrame(results)

Streaming output truncated to the last 5000 lines.
Finished preprocessing dataframe.
Finished forecasting.
Processing dataframe with multiple processes.
Finished preprocessing dataframe.
Finished forecasting.
Processing dataframe with multiple processes.
Finished preprocessing dataframe.
Finished forecasting.
Processing dataframe with multiple processes.
Finished preprocessing dataframe.
Finished forecasting.
Processing dataframe with multiple processes.
Finished preprocessing dataframe.
Finished forecasting.
Processing dataframe with multiple processes.
Finished preprocessing dataframe.
Finished forecasting.
Processing dataframe with multiple processes.
Finished preprocessing dataframe.
Finished forecasting.
Processing dataframe with multiple processes.
Finished preprocessing dataframe.
Finished forecasting.
Processing dataframe with multiple processes.
Finished preprocessing dataframe.
Finished forecasting.
Processing dataframe with multiple processes.
Finished preprocessing datafram

## Save results

In [ ]:
results_df.to_csv(f'/content/drive/MyDrive/TimesFM//Results_{var_int}_{portion_}.csv') #change this

#Perfromance Analysis

##NSE and KGE

In [ ]:
# Function to calculate Nash-Sutcliffe Efficiency (NSE)
def nse(observed, forecasted):
    numerator = np.sum((observed - forecasted) ** 2)
    denominator = np.sum((observed - np.mean(observed)) ** 2)
    return 1 - (numerator / denominator)

# Function to calculate Kling-Gupta Efficiency (KGE)
def kge(observed, forecasted):
    # Calculate correlation coefficient
    r = np.corrcoef(observed, forecasted)[0, 1]

    # Calculate variability ratio (alpha)
    alpha = np.std(forecasted) / np.std(observed)

    # Calculate bias ratio (beta)
    beta = np.mean(forecasted) / np.mean(observed)

    # Calculate KGE
    kge_value = 1 - np.sqrt((r - 1) ** 2 + (alpha - 1) ** 2 + (beta - 1) ** 2)
    return kge_value

# Assuming `results_df` has the observed and forecasted values for day 1 (h1)
# Calculate NSE and KGE for the one-day-ahead forecast (h1)
observed = results_df['obs'].values
forecasted_h1 = results_df['median'].values

nse_h1 = nse(observed, forecasted_h1)
kge_h1 = kge(observed, forecasted_h1)

# Print NSE and KGE values
print(f"Nash-Sutcliffe Efficiency (NSE) for 1st day forecast: {nse_h1}")
print(f"Kling-Gupta Efficiency (KGE) for 1st day forecast: {kge_h1}")

Nash-Sutcliffe Efficiency (NSE) for 1st day forecast: 0.6394291076662193
Kling-Gupta Efficiency (KGE) for 1st day forecast: 0.6284951823640756
